In [1]:
import pandas as pd
import os
import geopandas as gpd

In [2]:
folder_path = "E:/D-Drive/DW_intern_SATBIR/data_all/data_world_2021"

In [3]:

summary_data = []
all_data = [] 

In [4]:
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(folder_path, filename)
        df = pd.read_csv(filepath)
        all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)
combined_df = combined_df.drop_duplicates(subset=["HYBAS_ID", "Period"])

for hybas_id in combined_df["HYBAS_ID"].unique():
    df = combined_df[combined_df["HYBAS_ID"] == hybas_id]

    total_area = df["total_area"].iloc[0]

    monthly_df = df[~df["Period"].str.contains("total")]

    monthly_mean = monthly_df["Coverage_percent"].mean()
    monthly_min = monthly_df["Coverage_percent"].min()
    monthly_max = monthly_df["Coverage_percent"].max()
    monthly_variation = monthly_max - monthly_min

    yearly_row = df[df["Period"] == "2021_total"]
    yearly_value = yearly_row["Coverage_percent"].values[0] if not yearly_row.empty else None

    summary_data.append({
        "HYBAS_ID": hybas_id,
        "monthly_mean": monthly_mean,
        "monthly_min": monthly_min,
        "monthly_max": monthly_max,
        "monthly_variation": monthly_variation,
        "yearly_value": yearly_value,
        "total_area": total_area
    })
summary_df = pd.DataFrame(summary_data)


In [5]:
summary_df

,HYBAS_ID,monthly_mean,monthly_min,monthly_max,monthly_variation,yearly_value,total_area
0,1050011180,84.147979,30.864025,100.0,69.135975,100.0,1.546112e+06
1,1050011420,83.956548,28.495790,100.0,71.504210,100.0,2.408338e+06
2,1050011660,93.383375,75.478004,100.0,24.521996,100.0,1.177813e+06
3,1050011670,88.913246,31.172099,100.0,68.827901,100.0,1.866685e+06
4,1050011970,88.956534,40.810321,100.0,59.189679,100.0,1.067633e+05
...,...,...,...,...,...,...,...
859,9050011530,72.873978,0.000000,100.0,100.000000,100.0,2.455112e+06
860,9050011740,51.813129,0.000000,100.0,100.000000,100.0,2.261389e+07
861,9050014800,56.150486,0.000000,100.0,100.000000,100.0,3.266207e+07
862,9050014810,55.966851,0.000000,100.0,100.000000,100.0,1.343630e+07


In [6]:
summary_df.to_csv("E:/D-Drive/DW_intern_SATBIR/basin_summary/world_basins_summary_2021.csv", index=False)

In [ ]:
!pip install geopandas matplotlib

In [7]:
import geopandas as gpd

# Path to the GPKG file
gpkg_path = "E:/D-Drive/DW_intern_SATBIR/Hybas/HydroBASINS_lev_05.gpkg"

# Read the entire GPKG file
gdf_basins = gpd.read_file(gpkg_path)

print(f"Total basins loaded: {len(gdf_basins)}")
print(gdf_basins.columns)

Total basins loaded: 4734
Index(['OBJECTID', 'HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS',
       'DIST_SINK', 'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID',
       ...
       'hft_ix_s09', 'hft_ix_u09', 'gad_id_smj', 'gdp_ud_sav', 'gdp_ud_ssu',
       'gdp_ud_usu', 'hdi_ix_sav', 'Shape_Length', 'Shape_Area', 'geometry'],
      dtype='object', length=298)


In [8]:
gdf_basins["HYBAS_ID"] = gdf_basins["HYBAS_ID"].astype(int)
summary_df["HYBAS_ID"] = summary_df["HYBAS_ID"].astype(int)

# Merge GeoDataFrame with summary stats
gdf_merged = gdf_basins.merge(summary_df, on="HYBAS_ID", how="inner")

In [10]:
gdf_merged.shape

(864, 304)

In [11]:
# Export merged GeoDataFrame to GeoJSON for geemap
geojson_path = "E:/D-Drive/DW_intern_SATBIR/merged_geojson/merged_hydrobasins_World.geojson"
gdf_merged.to_file(geojson_path, driver="GeoJSON")

In [12]:

# Load your GeoDataFrame
gdf = gpd.read_file("E:/D-Drive/DW_intern_SATBIR/merged_geojson/merged_hydrobasins_World.geojson")

# Confirm column names
gdf.columns

Index(['OBJECTID', 'HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS',
       'DIST_SINK', 'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID',
       ...
       'hdi_ix_sav', 'Shape_Length', 'Shape_Area', 'monthly_mean',
       'monthly_min', 'monthly_max', 'monthly_variation', 'yearly_value',
       'total_area', 'geometry'],
      dtype='object', length=304)

In [13]:
import geopandas as gpd
import geemap
import ipywidgets as widgets
from ipyleaflet import GeoJSON, WidgetControl, Popup
from branca.colormap import linear
import matplotlib.pyplot as plt
from matplotlib import cm, colors 

def plot_choropleth(feature, cmap_name='viridis'):
    # Step 1: Clean data
    gdf_plot = gdf[["HYBAS_ID", "geometry", feature]].dropna()

    # Step 2: Colormap setup
    vmin, vmax = gdf_plot[feature].min(), gdf_plot[feature].max()
    colormap = linear.__getattribute__(cmap_name).scale(vmin, vmax)
    colormap.caption = feature.replace("_", " ").title()
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)
    
    # Step 3: Convert GeoDataFrame to styled GeoJSON
    styled_features = []
    id_lookup = {}
    for _, row in gdf_plot.iterrows():
        value = round(row[feature], 2)
        color = colormap(value)
        hybas_id = int(row["HYBAS_ID"])
        props = {
            "HYBAS_ID": hybas_id,
            feature: value,
            "style": {
                "color": "black",
                "weight": 0.5,
                "fillColor": color,
                "fillOpacity": 0.8
            }
        }
        id_lookup[hybas_id] = props  # Store for popup access
        styled_features.append({
            "type": "Feature",
            "geometry": row["geometry"].__geo_interface__,
            "properties": props
        })

    geojson_data = {
        "type": "FeatureCollection",
        "features": styled_features
    }

    gdf_plot["style"] = gdf_plot[feature].apply(lambda x: colors.to_hex(cmap(norm(x))))

    features_json = []
    for _, row in gdf_plot.iterrows():
        features_json.append({
            "type": "Feature",
            "geometry": row["geometry"].__geo_interface__,
            "properties": {
                "HYBAS_ID": row["HYBAS_ID"],
                feature: row[feature],
                "style": {
                    "color": "black",
                    "weight": 0.5,
                    "fillColor": row["style"],
                    "fillOpacity": 0.8
                }
            }
        })

    geojson_dict = {
        "type": "FeatureCollection",
        "features": features_json
    }
    # Step 4: Create the map
    m = geemap.Map(center=[22.7266, 74.9799], zoom=4)

    # Step 5: Add the GeoJSON layer
    geo_json_layer = GeoJSON(
        data=geojson_data,
        style={"opacity": 1, "fillOpacity": 0.7},
        hover_style={"fillColor": "white", "fillOpacity": 0.4},
        name=feature
    )
    m.add_layer(geo_json_layer)
    m.add_geojson(geojson_dict, layer_name=feature, info_mode="on_hover")

    # Step 7: Add color legend
    legend_html = colormap._repr_html_()
    legend_widget = widgets.HTML(value=legend_html)
    legend_control = WidgetControl(widget=legend_widget, position='bottomright')
    m.add_control(legend_control)


    return m


In [14]:
plot_choropleth("monthly_mean")

C:\Users\Satbir\AppData\Local\Temp\ipykernel_32392\958187023.py:18: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(cmap_name)


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [5]:
features = ["monthly_mean", "monthly_min", "monthly_max", "monthly_variation", "yearly_value"]

for feature in features:
    print(f"Interactive map for {feature}")
    display(plot_choropleth(feature))

Interactive map for monthly_mean


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Interactive map for monthly_min


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Interactive map for monthly_max


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Interactive map for monthly_variation


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Interactive map for yearly_value


Map(center=[22.7266, 74.9799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…